In [31]:
from google.colab import drive
import pandas as pd
import requests
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
# NOAA NCEI API 需要一个 API Key
API_TOKEN = "XbyuERiTrXrJMbExiMRwzCNOZKwjQQep"  # 需要去 https://www.ncdc.noaa.gov/cdo-web/token 申请

HEADERS = {
    "token": API_TOKEN
}

def get_noaa_station_id(lat, lon):
    """ 通过经纬度查询 NOAA 气象站 ID """
    # 定义查询范围（latitude +/- 0.5, longitude +/- 0.5 以找到最近站点）
    min_lat, max_lat = lat - 0.1, lat + 0.1
    min_lon, max_lon = lon - 0.1, lon + 0.1

    # 构造 API 请求 URL
    stations_url = f"https://www.ncdc.noaa.gov/cdo-web/api/v2/stations"
    params = {
        "extent": f"{min_lat},{min_lon},{max_lat},{max_lon}",  # 指定查询范围
        "datasetid": "GHCND",  # 选择 GHCND 数据集
        "limit": 5  # 限制返回站点数
    }

    response = requests.get(stations_url, headers=HEADERS, params=params)

    if response.status_code == 200:
        data = response.json()
        if "results" in data:
            stations = data["results"]
            for station in stations:
                print(f"找到气象站: {station['id']} - {station['name']}")
            return stations[0]['id']  # 返回最近的气象站 ID
        else:
            print("没有找到合适的气象站")
            return None
    else:
        print("请求 NOAA API 失败:", response.status_code, response.text)
        return None

In [32]:
df = pd.read_csv('/content/drive/MyDrive/Projects/Cherry Blossom/CODE/Liestal-Weideli/Data/japan.csv')

In [33]:
df.sample(5)

location        lat        long     alt  year  bloom_date  \
3561      Japan/Owase  34.069722  136.193333   15.30  1972  1972-04-03   
3357  Japan/Hamamatsu  34.753889  137.711667   45.93  1959  1959-04-03   
2170   Japan/Kanazawa  36.589722  136.634722    5.70  2007  2007-04-09   
3153       Japan/Kofu  35.667222  138.553889  272.80  1961  1961-04-06   
640     Japan/Muroran  42.323611  140.970000    3.00  1965  1965-05-20   

      bloom_doy  
3561         94  
3357         93  
2170         99  
3153         96  
640         140

In [34]:
df.columns

Index(['location', 'lat', 'long', 'alt', 'year', 'bloom_date', 'bloom_doy'], dtype='object')

In [35]:
df_lat_long = df[['location', 'lat', 'long']].copy()

In [37]:
df_lat_long.drop_duplicates(inplace=True)
df_lat_long

location        lat        long
0            Japan/Wakkanai  45.415000  141.678889
60              Japan/Rumoi  43.946111  141.631944
113         Japan/Asahikawa  43.756944  142.372222
182          Japan/Abashiri  44.017778  144.279722
242           Japan/Sapporo  43.060000  141.328611
...                     ...        ...         ...
6385       Japan/Miyakojima  24.794444  125.278333
6431         Japan/Kumejima  26.338056  126.803889
6462             Japan/Naha  26.207222  127.687222
6507             Japan/Nago  26.594167  127.966389
6530  Japan/Minamidaitojima  25.828889  131.228611

[111 rows x 3 columns]

In [38]:
df = df_lat_long.copy()

In [45]:
df.iloc[0]

location      wakkanai
lat             45.415
long        141.678889
Name: 0, dtype: object

In [40]:
df['location'] = df['location'].str.replace('Japan/', '')
df['location'] = df['location'].str.lower()

In [47]:
# 目标经纬度
latitude = 40.7128   # 纽约市
longitude = -74.0060

# 获取气象站 ID
station_id = get_noaa_station_id(latitude, longitude)
# 使用正则表达式提取 "NOAA 站点 ID:" 后面的字符
match = re.search(r"NOAA 站点 ID:\s*(\S+)", text)

# 输出结果
if match:
    station_id = match.group(1)
    print("提取的 NOAA 站点 ID:", station_id)
else:
    print("未找到 NOAA 站点 ID")


找到气象站: GHCND:US1NJHD0016 - SECAUCUS 0.8 NNE, NJ US
找到气象站: GHCND:US1NJHD0017 - SECAUCUS 1.2 NNE, NJ US
找到气象站: GHCND:US1NJHD0023 - JERSEY CITY 1.6 NNE, NJ US
找到气象站: GHCND:US1NYKN0003 - BROOKLYN 2.4 SW, NY US
找到气象站: GHCND:US1NYKN0025 - BROOKLYN 3.1 NW, NY US
NOAA 站点 ID: GHCND:US1NJHD0016


In [ ]:
import re

# 示例字符串
text = """找到气象站: GHCND:US1NJHD0016 - SECAUCUS 0.8 NNE, NJ US
找到气象站: GHCND:US1NJHD0017 - SECAUCUS 1.2 NNE, NJ US
找到气象站: GHCND:US1NJHD0023 - JERSEY CITY 1.6 NNE, NJ US
找到气象站: GHCND:US1NYKN0003 - BROOKLYN 2.4 SW, NY US
找到气象站: GHCND:US1NYKN0025 - BROOKLYN 3.1 NW, NY US
NOAA 站点 ID: GHCND:US1NJHD0016"""

# 使用正则表达式提取 "NOAA 站点 ID:" 后面的字符
match = re.search(r"NOAA 站点 ID:\s*(\S+)", text)

# 输出结果
if match:
    station_id = match.group(1)
    print("提取的 NOAA 站点 ID:", station_id)
else:
    print("未找到 NOAA 站点 ID")


In [48]:
# 目标经纬度
latitude = 45.415   # 纽约市
longitude = 141.678889

# 获取气象站 ID
station_id = get_noaa_station_id(latitude, longitude)
print("NOAA 站点 ID:", station_id)

请求 NOAA API 失败: 503 <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>503 Service Unavailable</title>
</head><body>
<h1>Service Unavailable</h1>
<p>The server is temporarily unable to service your
request due to maintenance downtime or capacity
problems. Please try again later.</p>
<p>Additionally, a 503 Service Unavailable
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>

NOAA 站点 ID: None


In [42]:
type(station_id)

str

In [11]:
import requests

# 设置 User-Agent 头，防止 403 错误
HEADERS = {
    "User-Agent": "Cherrybloomprediction (zn2209@columbia.com)"
}

def get_weather_forecast(latitude, longitude):
    # Step 1: 获取网格点信息
    points_url = f"https://api.weather.gov/points/{latitude},{longitude}"
    response = requests.get(points_url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Error fetching points data: {response.status_code}")
        return None

    points_data = response.json()
    forecast_url = points_data["properties"]["forecast"]

    # Step 2: 获取天气预报
    forecast_response = requests.get(forecast_url, headers=HEADERS)
    if forecast_response.status_code != 200:
        print(f"Error fetching forecast data: {forecast_response.status_code}")
        return None

    forecast_data = forecast_response.json()
    return forecast_data

# 示例：获取华盛顿纪念碑（Washington Monument）的天气预报
latitude = 38.8894
longitude = -77.0352
weather_forecast = get_weather_forecast(latitude, longitude)

if weather_forecast:
    for period in weather_forecast["properties"]["periods"]:
        print(f"{period['name']}: {period['detailedForecast']}")


Tonight: Mostly cloudy, with a low around 48. Southeast wind around 8 mph.
Thursday: Scattered rain showers after 7am. Cloudy. High near 69, with temperatures falling to around 63 in the afternoon. Southwest wind 9 to 16 mph, with gusts as high as 35 mph. Chance of precipitation is 50%. New rainfall amounts less than a tenth of an inch possible.
Thursday Night: Partly cloudy, with a low around 40. West wind 9 to 14 mph, with gusts as high as 25 mph.
Friday: Sunny, with a high near 55. West wind 9 to 14 mph, with gusts as high as 26 mph.
Friday Night: Partly cloudy, with a low around 43. South wind 8 to 12 mph.
Saturday: Partly sunny, with a high near 62.
Saturday Night: Partly cloudy, with a low around 28.
Sunday: Sunny, with a high near 41.
Sunday Night: Clear, with a low around 24.
Monday: Sunny, with a high near 44.
Monday Night: Mostly clear, with a low around 30.
Tuesday: A slight chance of rain showers after 1pm. Partly sunny, with a high near 56.
Tuesday Night: A chance of rain 

In [22]:
import requests

# 设置目标经纬度
latitude = 43.7128  # 例如纽约市
longitude = -74.0060

# NOAA API 请求头（必须包含 User-Agent）
HEADERS = {
    "User-Agent": "Cherrybloomprediction (zn2209@columbia.com)"
}

def get_station_id(lat, lon):
    """ 根据经纬度获取气象站 ID """
    # Step 1: 获取 `gridId`, `gridX`, `gridY`
    point_url = f"https://api.weather.gov/points/{lat},{lon}"
    response = requests.get(point_url, headers=HEADERS)

    if response.status_code == 200:
        data = response.json()
        wfo = data['properties']['gridId']
        x, y = data['properties']['gridX'], data['properties']['gridY']

        # Step 2: 通过 `gridpoints/{wfo}/{x},{y}/stations` 获取站点列表
        stations_url = f"https://api.weather.gov/gridpoints/{wfo}/{x},{y}/stations"
        response = requests.get(stations_url, headers=HEADERS)

        if response.status_code == 200:
            stations_data = response.json()
            station_id = stations_data['features'][0]['properties']['stationIdentifier']
            return stations_data
        else:
            print("无法获取气象站信息:", response.status_code)
            return None
    else:
        print("无法获取经纬度对应的气象数据:", response.status_code)
        return None

# 获取气象站 ID
station_id = get_station_id(latitude, longitude)
print("气象站 ID:", station_id)


气象站 ID: {'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld', {'@version': '1.1', 'wx': 'https://api.weather.gov/ontology#', 's': 'https://schema.org/', 'geo': 'http://www.opengis.net/ont/geosparql#', 'unit': 'http://codes.wmo.int/common/unit/', '@vocab': 'https://api.weather.gov/ontology#', 'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'}, 'city': 's:addressLocality', 'state': 's:addressRegion', 'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'}, 'bearing': {'@type': 's:QuantitativeValue'}, 'value': {'@id': 's:value'}, 'unitCode': {'@id': 's:unitCode', '@type': '@id'}, 'forecastOffice': {'@type': '@id'}, 'forecastGridData': {'@type': '@id'}, 'publicZone': {'@type': '@id'}, 'county': {'@type': '@id'}, 'observationStations': {'@container': '@list', '@type': '@id'}}], 'type': 'FeatureCollection', 'features': [{'id': 'https://api.weather.gov/stations/KGFL', 'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-73.61056, 43.3411

In [23]:
station_id

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'},
   'observationStations': {'@container': '@list', '@type': '@id'}}],
 'type': 'FeatureCollection',
 'features': [{'id': 'https://api.weather.gov/stations/KGFL',
   'type': 'Feature',
   'geometry':

In [6]:
station_id = ""

In [7]:
import datetime

def get_historical_weather(station_id, start_date, end_date):
    """ 获取历史天气数据 """
    base_url = f"https://api.weather.gov/stations/{station_id}/observations"
    params = {
        "start": start_date,
        "end": end_date
    }

    response = requests.get(base_url, headers=HEADERS, params=params)

    if response.status_code == 200:
        data = response.json()
        return data["features"]
    else:
        print("获取历史天气数据失败:", response.status_code)
        return None

# 设定时间范围（比如 2023 年 1 月 1 日 - 1 月 10 日）
start_date = "2023-01-10T00:00:00Z"
end_date = "2023-01-10T23:59:59Z"

# 获取历史天气数据
if station_id:
    weather_data = get_historical_weather(station_id, start_date, end_date)
    for record in weather_data:
        print(record["properties"]["timestamp"], record["properties"]["temperature"]["value"])

In [8]:
weather_data

[]